In [1]:
%run google_maps_api_functions.py

In [ ]:
# Create dataframe with location name, latitude, longitude.  
df_coordinates = pd.DataFrame(
    [["01", "50.87725045952269", "4.700713376547312"],
     ["02", "50.87667332973864", "4.700680922840582"],
     ["03", "50.875965600507456", "4.7002780085017495"],
     ["04", "50.87563410304464", "4.700197458227693"],
     ["05", "50.87464931652004", "4.699889649514398"],
     ["06", "50.87424051538546", "4.700098238567968"],
     ["07", "50.873946807679744", "4.700107077294445"],
     ["Vrijthof", "50.879041132675574", "4.701208864753383"]
    ], 
     columns = ["location_name", "lat", "lon"]
)

In [6]:
# Extract location names as list to be used for later naming of csv files. 
names = df_coordinates['location_name'].tolist()

In [17]:
# For each location, obtain url to send request to Google Maps API
urls = urls_locations(df_coordinates)

In [20]:
for j, url in enumerate(urls):
    # For each location (url), obtain ids of niglife places within a radius of 100 m. 
    ids = get_ids_places(url)
    
    # Initialize output
    df_info = pd.DataFrame()

    # For each nighlife place, obtain opening times. 
    for i, id in enumerate(ids):
        weekday_hours = get_week_hours(id) 

        # If the opening hours are available, clean them to be ready for further processing.
        if weekday_hours is not None: 
            info = openings(weekday_hours)
            df_info.loc[i, "info"] = info
        
        elif weekday_hours is None: 
            df_info.loc[i, "info"] = None
    
    # Save opening hours of nighlife places close to targe location in a csv for later analysis. 
    df_info.to_csv("data/location_%s.csv" %(names[j]), index=False)

    # Files were uploaded to s3 bucket: "s3://mda.project.monaco/location_%s.csv" %(location_id)


In [9]:
# Initialize output dataframe with counts of number of nighlife places that are open for all locations. 
df_out = pd.DataFrame()

# For eahc location, append dataframe with counts
for n in names: 
    df_out = pd.concat([df_out, count_open_bars(n)])

# Save df_out as csv
df_out.to_csv("data/openings.csv", index=False) 

# The file was uploaded to s3 bucket: s3://mda.project.monaco/openings.csv